In [42]:
import pandas as pd
from google.cloud import bigquery
import sys
from pathlib import Path
import os

sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/utils")
sys.path.append(str(Path(".").absolute().parent.parent))
sys.path.append(str(Path(".").absolute().parent.parent.parent))

import pipeline_utils
import argparse
import mlflow

In [21]:
PARAMS = pipeline_utils.yaml_import('settings.yml')

ENV = PARAMS['env_flag']

PROJECT_ID = PARAMS['envs'][ENV]['PROJECT_ID']

In [35]:
parser = argparse.ArgumentParser()
parser.add_argument("--MODE", required=True, type=str)
parser.add_argument("--STAGE1_FLAG", required=True, type=str)
parser.add_argument("--ENSEMBLE_FLAG", required=True, type=str)
parser.add_argument("--RF_CLF_MODEL_PATH", required=True, type=str)
parser.add_argument("--LOGISTIC_CLF_MODEL_PATH", required=True, type=str)
parser.add_argument("--STAGE1_NN_MODEL_PATH", required=True, type=str)
parser.add_argument("--GNB_MODEL_PATH", required=True, type=str)
parser.add_argument("--STG1_FEATURE_SELECTOR_MODEL_PATH", required=True, type=str)
parser.add_argument("--NOSALES_MODEL_PATH", required=True, type=str)
sys.args = [
    "--MODE", "test",
    "--STAGE1_FLAG", "train",
    "--ENSEMBLE_FLAG", "train",
    "--RF_CLF_MODEL_PATH", "",
    "--LOGISTIC_CLF_MODEL_PATH", "",
    "--STAGE1_NN_MODEL_PATH", "",
    "--GNB_MODEL_PATH", "",
    "--STG1_FEATURE_SELECTOR_MODEL_PATH", "",
    "--NOSALES_MODEL_PATH", "",
]
args = parser.parse_args(sys.args)

In [17]:
# result_path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221102171002/update-thresholds_-6212966924364546048/club_threshold_output"
# a = pd.read_csv(result_path)

c = bigquery.Client(project=PROJECT_ID)
nosales_test_ext_table = "oyi_test.downsampled_oyi_nosales_test_ext"
nosales_test_ext = c.query(f"SELECT * FROM {nosales_test_ext_table}").to_dataframe()

In [41]:
print(os.getenv('schema'))


None


In [71]:
!gsutil ls gs://oyi-ds-vertex-pipeline-bucket-nonprod

gs://oyi-ds-vertex-pipeline-bucket-nonprod/intro_20221017234643-dev.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev
gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_stage
gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_training_pipeline_dev.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_training_pipeline_stage.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/oyi-nosales-model-pipeline-dev_dev_08c758e7160a9dcc157bec22f0fb83ca5293a2b9.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/oyi-nosales-model-pipeline-dev_dev_130fb3eb2d9b58a3be6eeddf38a1c59bb734621e.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/oyi-nosales-model-pipeline-dev_dev_1544ee328759d357c0b0bdbf36cdf652a135b5f7.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/oyi-nosales-model-pipeline-dev_dev_1d9953c2e5a2ff2e7be84c7b320f9ac872a979f7.json
gs://oyi-ds-vertex-pipeline-bucket-nonprod/

In [61]:
!gsutil ls gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev

gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev


In [67]:
import tensorflow as tf
import joblib
import pickle
import pandas as pd
import pickle
import os
from google.cloud import storage
# from tempfile import TemporaryFile

nosales_model_path_input = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev"

path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221201223121/train-eval-model_-856660020647952384/nosales_model_output"

storage_client = storage.Client()

bucket = storage_client.bucket('oyi-ds-vertex-pipeline-bucket-nonprod')
blob = bucket.blob('latest_nosales_model_output_dev')
pickle_in = blob.download_as_string()
my_dictionary = pickle.load(pickle_in)

# # MODIFY DICTIONARY BY, FOR EXAMPLE, ADDING NEW REGISTERS

# pickle_out = pickle.dumps(my_modified_dictionary)
# blob.upload_from_string(pickle_out)


# stack_pipeline = pickle.load(path)
    


TypeError: file must have 'read' and 'readline' attributes

In [ ]:
nosales_thresh = utils.gen_thresholds(df = nosales_test_ext,  predictions = stack_pipeline.predict_proba(X=nosales_test_ext), classes = stack_pipeline.classes_)
df_nosales_thresh = pd.DataFrame(nosales_thresh.items(), columns = ['club_nbr','nosales_club_thresh'])

loaded_model = joblib.load(nosales_model_path_input) 
# loaded_model = joblib.load(tf.io.gfile.GFile(nosales_model_path_input, 'rb'))

In [48]:

from google.cloud import storage
# from sklearn.externals import joblib
import joblib

from tempfile import TemporaryFile

storage_client = storage.Client()
bucket_name="oyi-ds-vertex-pipeline-bucket-nonprod"
model_bucket="latest_nosales_model_output_dev"

bucket = storage_client.get_bucket(bucket_name)
#select bucket file
blob = bucket.blob(model_bucket)
with TemporaryFile() as temp_file:
    #download blob into temp file
    blob.download_to_file(temp_file)
    temp_file.seek(0)
    #load into joblib
    pickle.load(open(filename, 'rb'))
    model=joblib.load(temp_file)


AttributeError: module 'utils' has no attribute 'EnsembleClassifier'

In [ ]:
nosales_model_path_input = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev"


gcs_model_path = args.GCS_MODEL_PATH
model_name = args.MODEL_REGISTRY_NAME
experiment_name = args.MLFLOW_EXP_NAME


mlflow.set_experiment(experiment_name)

with mlflow.start_run():
    blob = storage.blob.Blob.from_string(gcs_model_path, client=storage.Client())
    with TemporaryFile() as temp_file:
        blob.download_to_file(temp_file)
        temp_file.seek(0)
        model=pickle.load(temp_file)
    mlflow.pyfunc.log_model(python_model=utils.EnsembleClassifierWrapper(model=model,), 
                            artifact_path="model",
                            registered_model_name=model_name)


In [74]:
!pip install file-io


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 23.8 MB/s eta 0:00:00
  Created wheel for aiofile: filename=aiofile-3.8.1-py3-none-any.whl size=19434 sha256=0c568dffdea6027d314a25ba69d009709100f2173a86a571f80b929a46e9e952
  Stored in directory: /home/jupyter/.cache/pip/wheels/ff/2e/eb/8a071ffd66fead648f4da7c14d7d6e231fc22e214cce2250c5
Successfully built aiofile


In [77]:
# !gsutil cat -h gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev

In [78]:
# from fileio import File

nosales_model_path_input = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev"
# pathlike = File(nosales_model_path_input)

import pickle
print("Run the loading python file.")
# path = "/Users/j0c0p72/Projects/Sams/oyi/latest_nosales_model_output_dev"
with open(nosales_model_path_input, "rb") as handler:
    stack_pipeline = pickle.load(handler)
    print("loaded model")
    
# with file_io.FileIO(report_name, mode='r') as input_f:
#     with file_io.FileIO(job_dir + '/' + report_name, mode='w+') as output_f:
#         output_f.write(input_f.read())
        
# # with open(nosales_model_path_input.path, "rb") as handler:
# #     stack_pipeline = pickle.load(handler)
    
# with open(nosales_model_path_input, "r") as model:
#     stack_pipeline = pickle.load(model)


Run the loading python file.


FileNotFoundError: [Errno 2] No such file or directory: 'gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev'

In [79]:
from google.cloud import storage

# Instantiates a client
c = storage.Client()

bucket_name = 'oyi-ds-vertex-pipeline-bucket-nonprod'

bucket = c.get_bucket(bucket_name)

blob = bucket.get_blob('latest_nosales_model_output_dev')

downloaded_blob = blob.download_as_string()

print(downloaded_blob)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [84]:
import utils


In [86]:
def read_model(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"
    from google.cloud import storage
    import utils
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
    # with blob.open("w") as f:
    #     f.write("Hello world")
    with blob.open("rb") as handler:
        stack_pipeline = pickle.load(handler)
        print("loaded model")
    
#     with blob.open("r") as f:
#         f.read()
bucket_name = "oyi-ds-vertex-pipeline-bucket-nonprod"
blob_name = "latest_nosales_model_output_dev"
read_model(bucket_name, blob_name)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
regularized_club_thresh_chain_path = "gs://oyi-ds-club-score-cutoff-pipeline-bucket-nonprod/335163835346/oyi-ds-club-score-cutoff-pipeline-bucket-dev-20221013005650/combine-results_-6831875422569889792/regularized_club_thresh_chain_output"
regularized_club_thresh_chain = pd.read_csv(regularized_club_thresh_chain_path)


In [ ]:
regularized_club_thresh_chain.head(3)

In [5]:
check_path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221122120157/data-preprocessing_2211829817236520960/training_data_output"
check_result = pd.read_csv(check_path)
check_result.head()

,club_nbr,item_nbr,old_nbr,days_no_sale,cat,subcat,state,unit_retail,oh_qty,avg_sales_interval,...,mkt_cat_no_action_taken_tpr,mkt_cat_add_to_picklist_tpr,mkt_cat_update_ohq_tpr,mkt_cat_update_loc_tpr,mkt_cat_new_price_sign_tpr,reg_cat_no_action_taken_tpr,reg_cat_add_to_picklist_tpr,reg_cat_update_ohq_tpr,reg_cat_update_loc_tpr,reg_cat_new_price_sign_tpr
0,4041,54136277,980130476,2.0,47,25,NJ,24.98,4,10.042373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4109,20815152,817534,1.0,41,66,LA,9.37,1,1.822526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4109,21845167,415604,7.0,19,49,LA,8.98,3,3.078261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4109,30391872,558295,2.0,56,90,LA,4.98,1,0.420483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4109,31455679,317336,1.0,49,1,LA,6.78,10,0.212132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
check_result.count()

club_nbr                       1376591
item_nbr                       1376591
old_nbr                        1376591
days_no_sale                   1376591
cat                            1376591
                                ...   
reg_cat_no_action_taken_tpr    1376591
reg_cat_add_to_picklist_tpr    1376591
reg_cat_update_ohq_tpr         1376591
reg_cat_update_loc_tpr         1376591
reg_cat_new_price_sign_tpr     1376591
Length: 82, dtype: int64

In [7]:
gcp_result_path = "gs://oyi-ds-club-score-cutoff-pipeline-bucket-nonprod/latest_club_thresh_chain_dev/club_thresh_chain.csv"
gcp_result = pd.read_csv(gcp_result_path)

In [8]:
gcp_result.head()

,club_nbr,cancelled_club_thresh,nosales_club_thresh
0,4041,0.5440,0.2648
1,4109,0.5617,0.5426
2,4702,0.5335,0.9891
3,4703,0.3861,0.3049
4,4704,0.3949,0.4040


In [9]:
gcp_result.count()

club_nbr                 599
cancelled_club_thresh    599
nosales_club_thresh      599
dtype: int64